# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.3/872.3 kB 22.7 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO
import os
from IPython.display import Image, display
from IPython import display
display.clear_output()

## Load the YOLO Model

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [ ]:
import cv2
from ultralytics import YOLO
import numpy as np
!zip -r /content/YOLO_Model.zip /content/YOLO_Model.pt
model = YOLO('/content/YOLO_Model.pt')

updating: content/YOLO_Model.pt (deflated 8%)


## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [ ]:
video_path = '/content/YOLO_Video.mp4'
cap = cv2.VideoCapture(video_path)

## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [ ]:
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

In [ ]:
print(width)
print(height)
print(fps)

1920
1080
29


## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [ ]:
output_path = '/content/output_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

In [ ]:
center_x, center_y

(1371, 624)

In [ ]:

color = (0, 255, 0)
thickness = 2
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.8
font_thickness = 1

# Set to track unique object IDs
unique_ids = set()

top_region = (400 ,300 , 401, 301)
bottom_region = (1000, 300, 1001, 301)
left_region = (600,910 , 601, 911)
right_region = (1385, 620 , 1386, 621)

exit_counts = {'top': 0, 'bottom': 0, 'left': 0, 'right': 0}
unique_ids_left = set()
unique_ids_right = set()

unique_ids_top = set()
unique_ids_bottom = set()
ret = True
while ret:
    ret, frame = cap.read()

    if not ret:
        break

    results = model.track(frame, persist=True)

    for result in results:
        for obj in result.boxes:
            bbox = obj.xyxy[0].cpu().numpy()
            class_id = int(obj.cls[0].cpu().numpy()) if obj.cls is not None else -1
            conf = obj.conf[0].cpu().numpy() if obj.conf is not None else 0.0
            obj_id = int(obj.id[0].cpu().numpy()) if obj.id is not None else -1

            unique_ids.add(obj_id)

            x1, y1, x2, y2 = map(int, bbox)
            center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2

            cv2.rectangle(frame, (x1, y1), (x2, y2), color, thickness)

            label = f'ID: {obj_id} Class: {class_id} Conf: {conf:.2f}'

            cv2.putText(frame, label, (x1, y1 - 10), font, font_scale, color, font_thickness, cv2.LINE_AA)





            if top_region[0] <= center_x <= top_region[2] and top_region[1] <= center_y <= top_region[3]:
                unique_ids_top.add(obj_id)
            # Check if the car is in the bottom ROI
            elif bottom_region[0] <= center_x <= bottom_region[2] and bottom_region[1] <= center_y <= bottom_region[3]:
                unique_ids_bottom.add(obj_id)
            elif left_region[0] <= center_x <= left_region[2] and left_region[1] <= center_y <= left_region[3]:
                unique_ids_left.add(obj_id)
            elif right_region[0] <= center_x <= right_region[2] and right_region[1] <= center_y <= right_region[3]:
                unique_ids_right.add(obj_id)

    # Print the unique object count at the bottom of the frame, top and total
    cv2.rectangle(frame, (top_region), (255, 0, 0), 2)  # Blue rectangle for top
    cv2.rectangle(frame, (bottom_region), (0, 255, 0), 2)  # Green rectangle for bottom
    cv2.rectangle(frame, (left_region), (0, 0, 255), 2)  # Red rectangle for left
    cv2.rectangle(frame, (right_region), (0, 255, 255), 2)  # Yellow rectangle for right

    frame_text = f"Top: {unique_ids_top} Bottom: {unique_ids_bottom} Left: {unique_ids_left} Right: {unique_ids_right}"
    cv2.putText(frame, frame_text, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
    name =  f'Rahaf Alharthi'
    cv2.putText(frame, name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
    out.write(frame)

cap.release()
out.release()

## Save and Submit Your Work

Submit both the notebook and the output video

https://drive.google.com/file/d/1ZnRHls4MIYEsfek6xbEOXn8zknYBIwqe/view?usp=sharing